# FEMA's National Flood Insurance Policy Database Data Exploration

### Load Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Imputer
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, ByteType, LongType, FloatType, ShortType
from pyspark.sql.functions import col, sum as spark_sum

### Initialize Spark Session

In [2]:
spark = SparkSession.builder \
    .config("spark.driver.memory", "10g") \
	.config("spark.executor.memory", "2g") \
    .config('spark.executor.instances', 5) \
	.config("spark.sql.debug.maxToStringFields", "100")\
    .appName("Flood Data") \
	.getOrCreate()

24/05/17 12:03:38 WARN Utils: Your hostname, Joyces-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.0.0.2 instead (on interface en0)
24/05/17 12:03:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/17 12:03:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Load data to Spark dataframe; Infer Schema

In [3]:
#path to data 
data_path = "./NFIP/nfip-flood-policies.csv"

df = spark.read.option("header","true").option("inferSchema","true").csv(data_path)

## Perform the Data Exploration
Below is the outline of the Data Exploration Section:
- Print Schema
- Column Description
- Display the Number of Variables (Columnns)
- Show the First Few Rows
- Display Number of Observations
- Missing Values in the Dataframe
- Statistics Summary and Data Distribution
- Correlations Among Variables
- Scatterplots

### Print Schema

Understanding the structure of data (column names and types) is crucial.

In [4]:
df.printSchema()

root
 |-- agriculturestructureindicator: string (nullable = true)
 |-- basefloodelevation: double (nullable = true)
 |-- basementenclosurecrawlspacetype: integer (nullable = true)
 |-- cancellationdateoffloodpolicy: date (nullable = true)
 |-- censustract: long (nullable = true)
 |-- condominiumindicator: string (nullable = true)
 |-- construction: string (nullable = true)
 |-- countycode: integer (nullable = true)
 |-- crsdiscount: double (nullable = true)
 |-- deductibleamountinbuildingcoverage: string (nullable = true)
 |-- deductibleamountincontentscoverage: string (nullable = true)
 |-- elevatedbuildingindicator: string (nullable = true)
 |-- elevationcertificateindicator: string (nullable = true)
 |-- elevationdifference: integer (nullable = true)
 |-- federalpolicyfee: integer (nullable = true)
 |-- floodzone: string (nullable = true)
 |-- hfiaasurcharge: integer (nullable = true)
 |-- houseofworshipindicator: string (nullable = true)
 |-- latitude: double (nullable = true)
 |--

### Column Description

**Description of the columns from the FEMA's National Flood Insurance Policy Database, grouped by their data types and purpose:**

**Geographic and Location Details**
- **censustract (long):** Census tract number indicating the specific area where the property is located, used for demographic analysis.
- **countycode (integer):** Numeric code representing the county in which the property is insured.
- **floodzone (string):** Designation of the flood zone according to FEMA's mapping, crucial for assessing the property's flood risk.
- **latitude (double), longitude (double):** Geographic coordinates specifying the precise location of the property.
- **propertystate (string):** The U.S. state where the property is located.
- **reportedcity (string):** The city reported for the insured property.
- **reportedzipcode (integer):** Zip code where the property is situated, used for localizing insurance coverage and risk.

**Property and Construction Details**
- **agriculturestructureindicator (string):** Indicates whether the property is used for agricultural purposes.
- **basementenclosurecrawlspacetype (integer):** Type of basement or crawlspace present at the property, affecting flood risk assessment.
- **construction (string):** Describes the type of construction materials and methods used, which can affect the property's vulnerability to flood damage.
- **numberoffloorsininsuredbuilding (integer):** Total floors in the insured building, important for determining potential flood damage and insurance coverage needs.
- **elevatedbuildingindicator (string):** Indicates whether the building is elevated, a key factor in reducing flood risk.

**Policy Details**
- **policycost (integer):** The total cost of the flood insurance policy.
- **policycount (integer):** The number of policies associated with a single property or account.
- **policyeffectivedate (date), policyterminationdate (date):** Start and end dates of the flood insurance coverage.
- **totalbuildinginsurancecoverage (integer), totalcontentsinsurancecoverage (integer):** The amount of insurance coverage for the building and its contents, respectively.
- **totalinsurancepremiumofthepolicy (integer):** Total premium amount for the flood insurance policy.

**Flood Risk Assessment Specifics**
- **basefloodelevation (double):** The base flood elevation expected for a particular area, critical for understanding flood risk levels.
- **elevationcertificateindicator (string), elevationdifference (integer):** Presence of an elevation certificate and the difference in elevation, respectively, both crucial for assessing compliance with floodplain management.
- **lowestadjacentgrade (double), lowestfloorelevation (double):** Measures of elevation that help determine the property's flood exposure.

**Insurance Policy Features**
- **crsdiscount (double):** Community Rating System discount applied to the policy, which can reduce insurance premiums based on community flood preparedness.
- **deductibleamountinbuildingcoverage (integer), deductibleamountincontentscoverage (integer):** Deductible amounts for building and contents coverage, influencing out-of-pocket costs after a flood.
- **hfiaasurcharge (integer):** Surcharge applied under the Homeowner Flood Insurance Affordability Act.
- **federalpolicyfee (integer):** A fee associated with the federal policy governing flood insurance.

**Special Indicators**
- **condominiumindicator (string), primaryresidenceindicator (string):** Indicate whether the insured property is a condominium or the primary residence of the owner.
- **houseofworshipindicator (string), nonprofitindicator (string):** Indicators of whether the property is used as a house of worship or is owned by a nonprofit organization, affecting policy terms and possibly qualifying for special considerations.
- **postfirmconstructionindicator (string):** Indicates if the building was constructed after the community's first Flood Insurance Rate Map was issued, which can affect insurance rates.
- **smallbusinessindicatorbuilding (string):** Indicates whether the insured building is used for small business purposes.

**Additional Policy and Coverage Information**
- **originalconstructiondate (date), originalnbdate (date):** Dates of original construction and the building's initial notebook entry, important for historical property assessments.
- **cancellationdateoffloodpolicy (date):** Date when the flood policy was cancelled, if applicable.
- **regularemergencyprogramindicator (string):** Indicates the type of FEMA program under which the policy is covered, distinguishing between regular and emergency management programs.
- **ratemethod (integer):** Describes the method used to calculate the insurance rate, impacting how premiums are determined.
- **locationofcontents (string):** Specifies where within the property the insured contents are located, relevant for claims and risk assessments.

### Display number of Variables (Columns)


In [5]:
num_variables = len(df.columns)
print("Number of Columns:", num_variables)

Number of Columns: 45


### Show the First Few Rows

In [6]:
df.show(5,vertical=True)

-RECORD 0--------------------------------------------------
 agriculturestructureindicator      | NULL                 
 basefloodelevation                 | NULL                 
 basementenclosurecrawlspacetype    | 2                    
 cancellationdateoffloodpolicy      | NULL                 
 censustract                        | 33013038500          
 condominiumindicator               | N                    
 construction                       | N                    
 countycode                         | 33013                
 crsdiscount                        | 0.0                  
 deductibleamountinbuildingcoverage | 0                    
 deductibleamountincontentscoverage | 0                    
 elevatedbuildingindicator          | N                    
 elevationcertificateindicator      | NULL                 
 elevationdifference                | 999                  
 federalpolicyfee                   | 13                   
 floodzone                          | X 

### Display Number of Observations

In [7]:
df.count()

50406943

FEMA's National Flood Insurance Policy Database, containing over 50 million (50,406,943) policy transactions.

### Missing Values in the Dataframe

In [8]:
#find number of missing values in the dataframe
missing_vals = df.select(*(spark_sum(col(i).isNull().cast("int")).alias(i) for i in df.columns))

In [9]:
missing_vals.show(vertical= True)

-RECORD 0--------------------------------------
 agriculturestructureindicator      | 38923313 
 basefloodelevation                 | 33636759 
 basementenclosurecrawlspacetype    | 802      
 cancellationdateoffloodpolicy      | 43614057 
 censustract                        | 467119   
 condominiumindicator               | 6        
 construction                       | 13       
 countycode                         | 48999    
 crsdiscount                        | 0        
 deductibleamountinbuildingcoverage | 661993   
 deductibleamountincontentscoverage | 5561584  
 elevatedbuildingindicator          | 258      
 elevationcertificateindicator      | 32606397 
 elevationdifference                | 0        
 federalpolicyfee                   | 0        
 floodzone                          | 169145   
 hfiaasurcharge                     | 0        
 houseofworshipindicator            | 34476251 
 latitude                           | 338699   
 locationofcontents                 | 15

**Here's a description of the missing value situation in the FEMA Flood Insurance Policy Database:** 

1. **High Missing Values:**
- **Base Flood Elevation, Latitude, Longitude, Lowest Adjacent Grade, Lowest Floor Elevation:** These fields each have around 50,406,943 missing values. This suggests a significant lack of geographic and elevation data, which are critical in flood insurance calculations.
- **Elevation Certificate Indicator, Elevation Difference:** Both fields are missing approximately 32,806,397 and 32,897,994 values respectively, indicating that elevation certificates, which are vital for verifying compliance with floodplain management regulations, are largely absent.
- **Obstruction Type:** Missing around 40,793,526 values, indicating that details about obstructions which can affect flood risk assessments are predominantly not reported.
- **House of Worship Indicator, Nonprofit Indicator:** Each has over 34,476,251 and 34,493,094 missing entries respectively, indicating a lack of identification of these property types, which might have different considerations in policy terms.

2. **Moderate Missing Values:**
- **Deductible Amount in Building Coverage, Deductible Amount in Contents Coverage:** Missing 15,649,149 and 18,265,104 values respectively, which implies incomplete data on policy deductibles that could affect premium calculations and risk assessments.
- **Location of Contents:** With 15,389,767 missing entries, there's substantial missing information on where contents are located within the insured buildings, which is vital for damage assessments.

3. **Low Missing Values:**
- **Census Tract, Flood Zone:** Missing 467,119 and 169,145 entries respectively. Although relatively lower, these still represent significant gaps, especially as these fields are crucial for location-specific risk assessment.
- **Number of Floors in Insured Building:** Missing data on 162,301 entries could affect understanding building structure and associated risk.

4. **Minimal to No Missing Values:**
- Fields like **CRS Discount, Federal Policy Fee**, and various policy-related dates (effectiveness, termination) and costs show zero missing values, indicating complete data in terms of policy transaction details.
- Similarly, **County Code, Construction, Condominium Indicator, Occupancy Type** show minimal missing data (under 50,000), suggesting good coverage of basic property and policyholder information.

Overall, the dataset shows a strong presence of policy and basic property information but suffers from a significant absence of detailed geographic and structural data. This gap in data can hamper effective risk assessment and pricing of flood insurance policies, especially in areas prone to flooding where such data is most critical. Addressing these missing values, either by data imputation where appropriate or by collecting missing data, could significantly enhance the robustness of any analysis or predictive modeling based on this dataset.

### Statistics Summary and Data Distribution

In [10]:
numerical_columns = [col_name for col_name, data_type in df.dtypes if data_type in ["int", "bigint", "double"]]

# Select only numerical columns
numerical_df = df.select(*numerical_columns)

# Generate summary statistics
summary_stats = numerical_df.describe()

# Show summary statistics
summary_stats.show(vertical = True)

24/05/17 12:06:27 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0------------------------------------------------
 summary                          | count                
 basefloodelevation               | 16770184             
 basementenclosurecrawlspacetype  | 50406141             
 censustract                      | 49939824             
 countycode                       | 50357944             
 crsdiscount                      | 50406943             
 elevationdifference              | 50406943             
 federalpolicyfee                 | 50406943             
 hfiaasurcharge                   | 50406943             
 latitude                         | 50068244             
 longitude                        | 50068244             
 lowestadjacentgrade              | 15466364             
 lowestfloorelevation             | 17346341             
 numberoffloorsininsuredbuilding  | 50244642             
 occupancytype                    | 50406937             
 policycost                       | 50406943             
 policycount  

The summary statistics for the FEMA National Flood Insurance Policy Database provide a comprehensive overview of various policy and property-related numerical attributes. These statistics include measures of central tendency, dispersion, and range, all of which are critical for understanding the distribution and potential data quality issues within the dataset. Below is a detailed analysis of the key statistical summaries:

**Central Tendency and Dispersion**
1. **Base Flood Elevation:**
- Average (Mean): 119.47 ft.
- Standard Deviation: 522.49 ft.
- Range: -9999 - 85,640 ft.
2. **Lowest Adjacent Grade:**
- Average (Mean): 129.20 ft.
- Standard Deviation: 609.92 ft.
- Range: -9,999 - 99,990.9 ft.
3. **Lowest Floor Elevation:**
- Average (Mean): 385.62 ft.
- Standard Deviation: 1,676.42 ft.
- Range: -9,997.9 - 99,989 ft.
4. **Basement Enclosure Crawl Space Type:**
- Average (Mean): 0.37, indicating a slight bias towards lower classifications.
- Standard Deviation: 0.86, showing moderate variability within the data.
- Range: Min 0 to Max 4, spanning several classification levels.
5. **Census Tract:**
- Average (Mean): Approximately 2.6 x 10¹⁰.
- Standard Deviation: About 1.58 x 10¹⁰, suggesting a wide spread across census tracts.
6. **CRS Discount:**
- Average: 0.064, typically low across the dataset.
- Standard Deviation: 0.091, with most data points close to zero but some higher values.
7. **Deductible Amount in Building and Contents Coverage:**
- Building Coverage Average: 1.66 with a deviation of 1.46.
- Contents Coverage Average: 0.98 with a deviation of 1.05.
- Both show low average deductible amounts but with notable variation.
8. **Elevation Difference:**
- Average: 1.69, indicating minor differences in elevation on average.
- Standard Deviation: 3.39, suggesting significant outliers affecting the elevation difference.
9. **Policy Related Figures (Policy Cost, Policy Count, Total Insurance Coverage, etc.):**
- These values have a high mean and standard deviation, indicating a significant spread in the policy costs and coverages, reflecting diverse insurance policies and property valuations.

**Extremes (Minimum and Maximum Values)**
- Notable minimums include negative values in **Federal Policy Fee and HFIAA Surcharge**, possibly indicating refunds or adjustments.
- The maximum values in **Total Building Insurance Coverage and Total Insurance Premium of the Policy** reach into the hundreds of millions, highlighting cases with exceptionally high insurance coverage.

**Implications**
The substantial missing data in critical geographical and elevation columns could significantly hinder risk assessment accuracy. The wide variability in policy costs and coverage levels underscores the diverse nature of the insured properties. Accurate and complete data in these fields are crucial for effective risk management and policy pricing in flood insurance.

This analysis provides a basis for further data cleaning, particularly in addressing missing values and outliers, which are essential for improving data quality and the reliability of subsequent analyses and decision-making processes based on this dataset.

### Correlations Among Variables

In [11]:
numerical_df.show(5,vertical=True)

-RECORD 0---------------------------------------
 basefloodelevation               | NULL        
 basementenclosurecrawlspacetype  | 2           
 censustract                      | 33013038500 
 countycode                       | 33013       
 crsdiscount                      | 0.0         
 elevationdifference              | 999         
 federalpolicyfee                 | 13          
 hfiaasurcharge                   | 0           
 latitude                         | 43.3        
 longitude                        | -71.8       
 lowestadjacentgrade              | NULL        
 lowestfloorelevation             | NULL        
 numberoffloorsininsuredbuilding  | 2           
 occupancytype                    | 1           
 policycost                       | 388         
 policycount                      | 1           
 policytermindicator              | 1           
 totalbuildinginsurancecoverage   | 250000      
 totalcontentsinsurancecoverage   | 100000      
 totalinsurancepremi

In [12]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Imputer

v_col = "features"
input = numerical_df.columns
output = [v_col + str(i) for i in range(len(input))]
imputer = Imputer(strategy='mean',inputCols=input,outputCols=output)
changed_df = imputer.fit(numerical_df).transform(numerical_df) 

assembler = VectorAssembler(inputCols=output,outputCol=v_col)
numerical_df_vector = assembler.transform(changed_df).select(v_col)
numerical_df_vector

DataFrame[features: vector]

In [13]:
matrix = Correlation.corr(numerical_df_vector,v_col).collect()[0][0]
corr_matrix = matrix.toArray().tolist()
print(corr_matrix)

24/05/17 12:09:38 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


[[1.0, 0.05835923233138656, 0.045348715060230874, 0.04452366797155514, -0.10226921108097828, 7.47005824308675e-05, -0.010387177126363124, 0.01428401512834972, 0.12638383851377266, -0.13551456752425806, 0.8181030501174288, 0.3047834007873046, 0.02123204991850198, 0.039488213849563546, -0.0017586364749227305, -0.008273477372144563, -1.657059310784373e-05, -0.007974781636194578, -0.0343224403726591, -0.002348467821387103], [0.05835923233138656, 1.0, 0.13623903586845487, 0.1322243889691115, -0.1337456840142641, 0.04144637455805757, 0.021515636223077703, 0.03920512967982893, 0.4322749154690187, 0.10160449532374921, 0.05012914035133893, -0.014776239600981919, 0.4205917981132781, -0.06389760287441464, 0.07213744564440354, 0.00879790297900046, -0.02450568557810686, 0.0069468869987788255, -0.09892408517021709, 0.0693259281247627], [0.045348715060230874, 0.13623903586845487, 1.0, 0.9861052821687085, -0.23633772590850222, 0.1406429179696707, -0.04651095355139787, -0.001706502567808509, 0.27126939

In [14]:
numerical_df.columns

['basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy']

In [15]:
columns = ['basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy']
df_c = spark.createDataFrame(corr_matrix,columns)

In [16]:
df_c.select('basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract').show(20)

+--------------------+-------------------------------+--------------------+
|  basefloodelevation|basementenclosurecrawlspacetype|         censustract|
+--------------------+-------------------------------+--------------------+
|                 1.0|            0.05835923233138656|0.045348715060230874|
| 0.05835923233138656|                            1.0| 0.13623903586845487|
|0.045348715060230874|            0.13623903586845487|                 1.0|
| 0.04452366797155514|             0.1322243889691115|  0.9861052821687085|
|-0.10226921108097828|            -0.1337456840142641|-0.23633772590850222|
| 7.47005824308675E-5|            0.04144637455805757|  0.1406429179696707|
|-0.01038717712636...|           0.021515636223077703|-0.04651095355139787|
| 0.01428401512834972|            0.03920512967982893|-0.00170650256780...|
| 0.12638383851377266|             0.4322749154690187| 0.27126939137487294|
|-0.13551456752425806|            0.10160449532374921| 0.03869302136198096|
|  0.8181030

In [17]:
df_c.select('countycode',
 'crsdiscount',
 'elevationdifference').show(20)

+--------------------+--------------------+--------------------+
|          countycode|         crsdiscount| elevationdifference|
+--------------------+--------------------+--------------------+
| 0.04452366797155514|-0.10226921108097828| 7.47005824308675E-5|
|  0.1322243889691115| -0.1337456840142641| 0.04144637455805757|
|  0.9861052821687085|-0.23633772590850222|  0.1406429179696707|
|                 1.0|-0.23953380471436225| 0.13888862687868733|
|-0.23953380471436225|                 1.0| -0.5259678054689083|
| 0.13888862687868733| -0.5259678054689083|                 1.0|
|-0.04591061242264458| 0.12618095833291212|-0.12308129153346878|
|-0.00380940314362...|0.048954413784206216|-0.04971959541878701|
|  0.2508501234655084| -0.2750855881555222|  0.1959940803211799|
| 0.04963240625202723| 0.12376428198089935| -0.1495597109740734|
| 0.03410361028464174|-0.08582096552979489|0.002425271072278...|
|-6.80664449099433...|-0.00575452231836...|0.005525565608556751|
|  0.1266704346367964|-0.

In [18]:
df_c.select('federalpolicyfee',
 'hfiaasurcharge',
 'latitude').show(20)

+--------------------+--------------------+--------------------+
|    federalpolicyfee|      hfiaasurcharge|            latitude|
+--------------------+--------------------+--------------------+
|-0.01038717712636...| 0.01428401512834972| 0.12638383851377266|
|0.021515636223077703| 0.03920512967982893|  0.4322749154690187|
|-0.04651095355139787|-0.00170650256780...| 0.27126939137487294|
|-0.04591061242264458|-0.00380940314362...|  0.2508501234655084|
| 0.12618095833291212|0.048954413784206216| -0.2750855881555222|
|-0.12308129153346878|-0.04971959541878701|  0.1959940803211799|
|                 1.0| 0.15998037490116276|-0.02569128904685...|
| 0.15998037490116276|                 1.0| 0.04994652724067947|
|-0.02569128904685...| 0.04994652724067947|                 1.0|
| 0.03643340484637453|0.011012441102289194| -0.1082262506822412|
|-0.00935471628409477|0.011091913551252462| 0.10842310185825517|
|-0.01125795805632...|-0.01874631398887...| 0.01787552000870374|
| 0.10729663798512859| 0.

In [19]:
df_c.select('longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation').show(20)

+--------------------+--------------------+--------------------+
|           longitude| lowestadjacentgrade|lowestfloorelevation|
+--------------------+--------------------+--------------------+
|-0.13551456752425806|  0.8181030501174288|  0.3047834007873046|
| 0.10160449532374921| 0.05012914035133893|-0.01477623960098...|
| 0.03869302136198096| 0.03463261496948558|1.374835906710930...|
| 0.04963240625202723| 0.03410361028464174|-6.80664449099433...|
| 0.12376428198089935|-0.08582096552979489|-0.00575452231836...|
| -0.1495597109740734|0.002425271072278...|0.005525565608556751|
| 0.03643340484637453|-0.00935471628409477|-0.01125795805632...|
|0.011012441102289194|0.011091913551252462|-0.01874631398887...|
| -0.1082262506822412| 0.10842310185825517| 0.01787552000870374|
|                 1.0|-0.12337700047640353|-0.04205996452248...|
|-0.12337700047640353|                 1.0|  0.2693569000679416|
|-0.04205996452248...|  0.2693569000679416|                 1.0|
|  0.1276384494270046| 0.

In [20]:
df_c.select('numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost').show(20)

+-------------------------------+--------------------+--------------------+
|numberoffloorsininsuredbuilding|       occupancytype|          policycost|
+-------------------------------+--------------------+--------------------+
|            0.02123204991850198|0.039488213849563546|-0.00175863647492...|
|             0.4205917981132781|-0.06389760287441464| 0.07213744564440354|
|            0.13092070392149213|-0.02151854477004...|-0.00940672579516...|
|             0.1266704346367964|-0.02083336431546965|-0.01020270930151...|
|           -0.06978076159056372| 0.04701164748485699|  0.0673989418393778|
|           -0.02959368105409...|-0.05068082979967...|6.468266631658918E-4|
|            0.10729663798512859| 0.14237367075146548| 0.38647213373301115|
|            0.05622556787460689|  0.3014557593894687| 0.18235192862173988|
|             0.3437598815781546|0.037078977440137384| 0.06080599667465571|
|             0.1276384494270046|-0.00248049165686...|0.025262116267291294|
|           

In [21]:
df_c.select('policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage').show(20)

+--------------------+--------------------+------------------------------+
|         policycount| policytermindicator|totalbuildinginsurancecoverage|
+--------------------+--------------------+------------------------------+
|-0.00827347737214...|-1.65705931078437...|          -0.00797478163619...|
| 0.00879790297900046|-0.02450568557810686|          0.006946886998778...|
|-0.02354252110006...|0.035625079186622007|          -0.02664276057268...|
|-0.02335026782739592| 0.04136907348096722|          -0.02736003235722...|
| 0.02814710877178117|-0.03970568654901...|           0.02521212997788549|
|-0.03065010725766519| 0.04104422570965058|          -0.03866066385828043|
|  0.5481074038104508|-0.02839057493420...|            0.4963566086681794|
|0.031949873197437695|-0.02571907189115...|           0.03871098206689112|
|-0.02512727932134053|0.001390069725837...|          -0.02446116456239017|
|0.010198719294559778|0.005318202524779173|          0.007148866689062...|
|-0.00739083457659...|-1.

In [22]:
df_c.select('totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy').show(20)

+------------------------------+--------------------------------+
|totalcontentsinsurancecoverage|totalinsurancepremiumofthepolicy|
+------------------------------+--------------------------------+
|           -0.0343224403726591|            -0.00234846782138...|
|          -0.09892408517021709|              0.0693259281247627|
|           0.01404538333090154|            -0.00788805859775...|
|          0.008361075386161178|            -0.00851191616326...|
|           -0.1443948051727502|            0.058638703250568816|
|           0.11507960281910591|            0.009922795564581028|
|          -0.09552873635534823|              0.3359488202322962|
|          0.014049042681092581|             0.11148432510815016|
|          -0.10442024470081457|             0.06066938471879821|
|          -0.00173734949061...|            0.023512821648468855|
|          -0.03205674352377111|            -0.00295009384701...|
|          -0.00959618414729...|            -0.01006926417078...|
|         

The dataframe new_df is the dataframe with the imputed mean values and this code checks that there are zero missing values and the column names are changed from feature0, feature1 and so on to the original column names within the new_df dataframe.

In [23]:
new_df = changed_df.drop('basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy')
new_df.show(5,vertical=True)

-RECORD 0------------------------
 features0  | 119.46875184553842 
 features1  | 2                  
 features2  | 33013038500        
 features3  | 33013              
 features4  | 0.0                
 features5  | 999                
 features6  | 13                 
 features7  | 0                  
 features8  | 43.3               
 features9  | -71.8              
 features10 | 129.1970600006573  
 features11 | 385.6220114835789  
 features12 | 2                  
 features13 | 1                  
 features14 | 388                
 features15 | 1                  
 features16 | 1                  
 features17 | 250000             
 features18 | 100000             
 features19 | 375                
-RECORD 1------------------------
 features0  | 119.46875184553842 
 features1  | 0                  
 features2  | 22063040700        
 features3  | 22063              
 features4  | 0.05               
 features5  | 999                
 features6  | 35                 
 features7  | 

In [24]:
new_df_missing_vals = new_df.select(*(spark_sum(col(i).isNull().cast("int")).alias(i) for i in new_df.columns))

In [25]:
new_df_missing_vals.show(vertical= True)

-RECORD 0---------
 features0  | 0   
 features1  | 0   
 features2  | 0   
 features3  | 0   
 features4  | 0   
 features5  | 0   
 features6  | 0   
 features7  | 0   
 features8  | 0   
 features9  | 0   
 features10 | 0   
 features11 | 0   
 features12 | 0   
 features13 | 0   
 features14 | 0   
 features15 | 0   
 features16 | 0   
 features17 | 0   
 features18 | 0   
 features19 | 0   



In [26]:
column_names = ['basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage',
 'totalinsurancepremiumofthepolicy']
new_df = new_df.toDF(*column_names)
new_df.show(5,vertical=True)

-RECORD 0----------------------------------------------
 basefloodelevation               | 119.46875184553842 
 basementenclosurecrawlspacetype  | 2                  
 censustract                      | 33013038500        
 countycode                       | 33013              
 crsdiscount                      | 0.0                
 elevationdifference              | 999                
 federalpolicyfee                 | 13                 
 hfiaasurcharge                   | 0                  
 latitude                         | 43.3               
 longitude                        | -71.8              
 lowestadjacentgrade              | 129.1970600006573  
 lowestfloorelevation             | 385.6220114835789  
 numberoffloorsininsuredbuilding  | 2                  
 occupancytype                    | 1                  
 policycost                       | 388                
 policycount                      | 1                  
 policytermindicator              | 1           

In [27]:
print(new_df.dtypes)

[('basefloodelevation', 'double'), ('basementenclosurecrawlspacetype', 'int'), ('censustract', 'bigint'), ('countycode', 'int'), ('crsdiscount', 'double'), ('elevationdifference', 'int'), ('federalpolicyfee', 'int'), ('hfiaasurcharge', 'int'), ('latitude', 'double'), ('longitude', 'double'), ('lowestadjacentgrade', 'double'), ('lowestfloorelevation', 'double'), ('numberoffloorsininsuredbuilding', 'int'), ('occupancytype', 'int'), ('policycost', 'int'), ('policycount', 'int'), ('policytermindicator', 'int'), ('totalbuildinginsurancecoverage', 'int'), ('totalcontentsinsurancecoverage', 'int'), ('totalinsurancepremiumofthepolicy', 'int')]


In [29]:
# 5/16 JOYCE: Construct Linear Regression model to perform Feature Importance (find weights of features, choose top weights' features to keep)
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

# creating train and test sets
# X = new_df.drop('totalinsurancepremiumofthepolicy')
# y = df['totalinsurancepremiumofthepolicy']
# ASK: Is 'features' correct?
assembler = VectorAssembler(
    inputCols=['basefloodelevation',
 'basementenclosurecrawlspacetype',
 'censustract',
 'countycode',
 'crsdiscount',
 'elevationdifference',
 'federalpolicyfee',
 'hfiaasurcharge',
 'latitude',
 'longitude',
 'lowestadjacentgrade',
 'lowestfloorelevation',
 'numberoffloorsininsuredbuilding',
 'occupancytype',
 'policycost',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage'],
    outputCol = 'features')
new_df = assembler.transform(new_df)
final_data = new_df.select('features', 'totalinsurancepremiumofthepolicy')

train_data, test_data = final_data.randomSplit([0.7, 0.3], seed=42)

lr = LinearRegression(featuresCol = 'features', labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium', regParam = 0.3)
lr_model = lr.fit(train_data)

24/05/17 12:25:40 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [36]:
# 5/16 JOYCE: Print coefficients of the features (will run this cell AFTER LinReg is successful)
coefficients = lr_model.coefficients
print("Coefficients: {}".format(lr_model.coefficients)) 

Coefficients: [-0.0003436622598299765,-2.7028946217449032,-3.126637110143797e-10,0.00020196885521779178,-28.575181212930342,-0.003544319828584374,-1.148271437106802,-1.4080199041381372,0.2798427203465903,0.05025740009204748,-0.001016993752871259,-0.0004107191911328043,0.9157052648294008,3.982258051665689,0.9342110367211142,3.967607045058722,7.06963423646242,-1.4447964502432627e-05,-1.3433801266527966e-05]


In [37]:
feature_importance = sorted(list(zip(new_df.columns[:-1], map(abs, coefficients))), key=lambda x: x[1], reverse=True)

print("Feature Importance:")
for feature, importance in feature_importance:
    print("  {}: {:.3f}".format(feature, importance))

Feature Importance:
  crsdiscount: 28.575
  policytermindicator: 7.070
  occupancytype: 3.982
  policycount: 3.968
  basementenclosurecrawlspacetype: 2.703
  hfiaasurcharge: 1.408
  federalpolicyfee: 1.148
  policycost: 0.934
  numberoffloorsininsuredbuilding: 0.916
  latitude: 0.280
  longitude: 0.050
  elevationdifference: 0.004
  lowestadjacentgrade: 0.001
  lowestfloorelevation: 0.000
  basefloodelevation: 0.000
  countycode: 0.000
  totalbuildinginsurancecoverage: 0.000
  totalcontentsinsurancecoverage: 0.000
  censustract: 0.000


In [ ]:
# 5/17 JOYCE: Construct Linear Regression prediction model with the new Chosen 10 Features

assembler_2 = VectorAssembler(
    inputCols=['basementenclosurecrawlspacetype',
 'censustract',
 'crsdiscount',
 'federalpolicyfee',
 'hfiaasurcharge',
 'occupancytype',
 'policycount',
 'policytermindicator',
 'totalbuildinginsurancecoverage',
 'totalcontentsinsurancecoverage'],
    outputCol = 'predict_features')

# new_df = assembler_2.transform(new_df)

new_final_data = new_df.select('predict_features', 'totalinsurancepremiumofthepolicy')
train_data, test_data = new_final_data.randomSplit([0.7, 0.3], seed=42)

lr_2 = LinearRegression(featuresCol = 'predict_features', labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_stats', regParam = 0.3)
lr_model_2 = lr_2.fit(train_data)
# IllegalArgumentException: predicted_premium_stats does not exist. For finding RMSE and r2 

In [ ]:
# 5/17 Get Training data RMSE for Model (IDK if this is using all 19 features or the 10 chosen, will review once model runs)
# RMSE ranges from 0 to infinity, lower RMSE the better, (higher weight to larger errors)

predictions_rmse_train = lr_model_2.transform(train_data)

evaluator = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_stats', metricName = 'rmse')
rmse_train = evaluator.evaluate(predictions_rmse_train)
print("Root Mean Squared Error (RMSE) on train data: {:.3f}".format(rmse_train))

In [ ]:
# 5/17 Get Training data R-squared for Model (IDK if this is using all 19 features or the 10 chosen, will review once model runs)

evaluator_r2 = RegressionEvaluator(labelCol='totalinsurancepremiumofthepolicy', predictionCol='predicted_premium', metricName='r2')
r2 = evaluator_r2.evaluate(predictions_rmse_train)
print("R-squared (R2) on train data: {:.3f}".format(r2))

In [47]:
# 5/17 Get Testing data RMSE for Model (IDK if this is using all 19 features or the 10 chosen, will review once model runs)
from pyspark.ml.evaluation import RegressionEvaluator

predictions_rmse_test = lr_model.transform(test_data)

evaluator = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_stats', metricName = 'rmse')
rmse_test = evaluator.evaluate(predictions_rmse_test)
print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(rmse_test))

In [48]:
# 5/17 Get Testing data R-squared for Model (IDK if this is using all 19 features or the 10 chosen, will review once model runs)

# evaluator_r2 = RegressionEvaluator(labelCol = 'totalinsurancepremiumofthepolicy', predictionCol = 'predicted_premium_stats', metricName = 'r2')
# r2 = evaluator_r2.evaluate(predictions_rmse_test)
# print("R-squared (R2) on test data: {:.3f}".format(r2))

In [ ]:
# 5/17 JOYCE: Scatter plot 
plt.figure(figsize = (10,10))
plt.scatter(true_value, predicted_value, c = 'crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(predicted_value), max(true_value))
p2 = min(min(predicted_value), min(true_value))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

### Scatterplots 

In [ ]:
import matplotlib.pyplot as plt
changed_num_df = changed_df.sample(False, 0.1)
num_pdf = changed_num_df.toPandas()
print(num_pdf)

In [ ]:
plt.scatter(x=num_pdf['policycost'],y=num_pdf['totalinsurancepremiumofthepolicy'])
plt.xlabel('Policy Cost')
plt.ylabel('Total Insurance Premium of the Policy')          
plt.title('Policy Cost vs Total Insurance Premium of the Policy')
plt.show()

In [ ]:
plt.scatter(x=num_pdf['countycode'],y=num_pdf['censustract'])
plt.xlabel('County Code')
plt.ylabel('Census Tract') 
plt.title('County Code vs Census Tract')
plt.show()

In [ ]:
plt.scatter(x=num_pdf['policycount'],y=num_pdf['totalbuildinginsurancecoverage'])
plt.xlabel('Policy Count')
plt.ylabel('Total Building Insurance Coverage') 
plt.title('Policy Count vs Total Building Insurance Coverage')
plt.show()